# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Math

# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Import your own code
import inauguralproject.py

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


NameError: name 'w2A' is not defined

# Question 1

**Illustration of set $\mathcal{C}$ in Edgeworth box**

In [5]:
# code for solving the model (remember documentation and comments)

a = np.array([1,2,3])
b = inauguralproject.square(a)
print(b)

NameError: name 'inauguralproject' is not defined

# Question 2

Calculation of error in the market clearing condition s

In [6]:
# code

# Question 3

Market clearing price



In [3]:
# code

# Question 4

4a) Allocation when only $\mathcal{P}_1$ can be chosen

In [4]:
# code

4b) Allocation when any positive price can be chosen

In [ ]:
# code

# Question 5

5a) Allocation if choice set is restricted to $\mathcal{C}$

In [ ]:
# code

5b) Allocation with no restrictions

In [ ]:
# code

# Question 6


6a) Allocation with social planner

In [ ]:
# code

6b) Illustration and comparison with previous questions

In [ ]:
# code

# Question 7

Set, $\mathcal{W}$, drawn with 50 elements

In [ ]:
# code

# Question 8

Market equilibrium allocation for each $\omega^A \in \mathcal{C}$ plotted in Edgeworth box

In [ ]:
# code
